# Importing library

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import time
import pandas_datareader.data as web
import yfinance as yf
from yahoofinancials import YahooFinancials
import talib

## Generating function for creating SMA and direction(Momentum)

In [31]:
# A function for calculating the SMA
def movingaverage(x, window):
    x[f'SMA'+str(window)]=x['Close'].rolling(window=window).mean()
    return x

In [48]:
def momentum(df,stock, day):
    df[str(stock)+'_momentum']=df['daily_close_change'].apply(lambda x:1 if x>=0 else 0)
    if day>1:
        df[str(stock)+str(day)+'_day_momentum']=df[str(stock)+'_momentum'].rolling(window=day).sum()
    elif day<1:
        print('The window cannot be negative')
    return df

# AAPL as an example

## Data cleaning and feature engineering

In [89]:
AAPL_df = yf.download('AAPL', 
                      start='2016-01-01', 
                      end='2020-12-20', 
                      progress=False)
AAPL_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-11,122.430000,122.760002,120.550003,122.410004,122.410004,86939800
2020-12-14,122.599998,123.349998,121.540001,121.779999,121.779999,79184500
2020-12-15,124.339996,127.900002,124.129997,127.879997,127.879997,157572300
2020-12-16,127.410004,128.369995,126.559998,127.809998,127.809998,98208600
2020-12-17,128.899994,129.580002,128.039993,128.699997,128.699997,94215500


In [90]:
AAPL_df['daily_close_change']=AAPL_df['Close'].diff()/AAPL_df['Close']

In [91]:
momentum(AAPL_df,'AAPL',1)

,Open,High,Low,Close,Adj Close,Volume,daily_close_change,AAPL_momentum
Date,,,,,,,,
2016-01-04,25.652500,26.342501,25.500000,26.337500,24.400942,270597600,NaN,0
2016-01-05,26.437500,26.462500,25.602501,25.677500,23.789471,223164000,-0.025703,0
2016-01-06,25.139999,25.592501,24.967501,25.174999,23.323915,273829600,-0.019960,0
2016-01-07,24.670000,25.032499,24.107500,24.112499,22.339539,324377600,-0.044064,0
2016-01-08,24.637501,24.777500,24.190001,24.240000,22.457672,283192000,0.005260,1
...,...,...,...,...,...,...,...,...
2020-12-11,122.430000,122.760002,120.550003,122.410004,122.410004,86939800,-0.006780,0
2020-12-14,122.599998,123.349998,121.540001,121.779999,121.779999,79184500,-0.005173,0
2020-12-15,124.339996,127.900002,124.129997,127.879997,127.879997,157572300,0.047701,1


In [92]:
#Shifting the momentum for testing the direction in the next trading day.
AAPL_df['AAPL_momentum']=AAPL_df['AAPL_momentum'].shift(-1)
AAPL_df

,Open,High,Low,Close,Adj Close,Volume,daily_close_change,AAPL_momentum
Date,,,,,,,,
2016-01-04,25.652500,26.342501,25.500000,26.337500,24.400942,270597600,NaN,0.0
2016-01-05,26.437500,26.462500,25.602501,25.677500,23.789471,223164000,-0.025703,0.0
2016-01-06,25.139999,25.592501,24.967501,25.174999,23.323915,273829600,-0.019960,0.0
2016-01-07,24.670000,25.032499,24.107500,24.112499,22.339539,324377600,-0.044064,1.0
2016-01-08,24.637501,24.777500,24.190001,24.240000,22.457672,283192000,0.005260,1.0
...,...,...,...,...,...,...,...,...
2020-12-11,122.430000,122.760002,120.550003,122.410004,122.410004,86939800,-0.006780,0.0
2020-12-14,122.599998,123.349998,121.540001,121.779999,121.779999,79184500,-0.005173,1.0
2020-12-15,124.339996,127.900002,124.129997,127.879997,127.879997,157572300,0.047701,0.0


In [93]:
# Generate the SMA10, SMA20 and SMA50. This can be changed to any numbers you want as long as it is resonable. Not suggest 200
#since it would eat up too many data. 
movingaverage(AAPL_df, 10)
movingaverage(AAPL_df, 20)
movingaverage(AAPL_df, 50)

,Open,High,Low,Close,Adj Close,Volume,daily_close_change,AAPL_momentum,SMA10,SMA20,SMA50
Date,,,,,,,,,,,
2016-01-04,25.652500,26.342501,25.500000,26.337500,24.400942,270597600,NaN,0.0,NaN,NaN,NaN
2016-01-05,26.437500,26.462500,25.602501,25.677500,23.789471,223164000,-0.025703,0.0,NaN,NaN,NaN
2016-01-06,25.139999,25.592501,24.967501,25.174999,23.323915,273829600,-0.019960,0.0,NaN,NaN,NaN
2016-01-07,24.670000,25.032499,24.107500,24.112499,22.339539,324377600,-0.044064,1.0,NaN,NaN,NaN
2016-01-08,24.637501,24.777500,24.190001,24.240000,22.457672,283192000,0.005260,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-11,122.430000,122.760002,120.550003,122.410004,122.410004,86939800,-0.006780,0.0,122.560001,120.010000,117.7474
2020-12-14,122.599998,123.349998,121.540001,121.779999,121.779999,79184500,-0.005173,1.0,122.833000,120.136000,117.9226
2020-12-15,124.339996,127.900002,124.129997,127.879997,127.879997,157572300,0.047701,0.0,123.349000,120.514999,118.1502


In [99]:
#Get rid off the last day for final testing.
final_test=AAPL_df.values[-1].reshape(1,len(AAPL_df.values[-1]))
final_test

array([[1.28899994e+02, 1.29580002e+02, 1.28039993e+02, 1.28699997e+02,
        1.28699997e+02, 9.42155000e+07, 6.91530234e-03,            nan,
        1.24397999e+02, 1.21469499e+02, 1.18715600e+02]])

## Scale the data with minmax scalar

In [24]:
#Create the dependent and independent variables. 
AAPL_X=AAPL_df.drop('AAPL_momentum', axis=1)
AAPL_y=AAPL_df['AAPL_momentum']

In [60]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn import preprocessing

In [36]:
scaler=preprocessing.MinMaxScaler()
scaler.fit(AAPL_X)
AAPL_scaled_X=scaler.transform(AAPL_X)

## Time series split and cross validation for random forest

In [48]:
tscv = TimeSeriesSplit(n_splits=5)
i = 1
score = []
for train_index, test_index in tscv.split(AAPL_X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test =  AAPL_scaled_X[train_index], AAPL_scaled_X[test_index]
    y_train, y_test = AAPL_y.iloc[train_index], AAPL_y.iloc[test_index]
    criterion=['gini', 'entropy']
    for ne in np.linspace(50, 100, 6):
        for md in np.linspace(20, 40, 5):
            for msl in np.linspace(30, 100, 8):
                for c in criterion:
                    rfc = RandomForestClassifier(
                        criterion=c,
                        n_estimators=int(ne),
                        max_depth=int(md),
                        min_samples_leaf=int(msl))
                    rfc.fit(X_train, y_train)
                    score.append([i,
                                    c, 
                                    ne,
                                    md, 
                                    msl, 
                                    rfc.score(X_test, y_test)])
    i += 1

## Get the optimal hyperparameter

In [51]:
max_score=0
parameter=[]
for i in range(len(score)):
    if score[i][5]>=max_score:
        max_score=score[i][5]
        parameter=score[i]

## Make prediction with the most recent data

In [81]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
rfc = RandomForestClassifier(
                    criterion='entropy',
                    n_estimators=50,
                    max_depth=25,
                    min_samples_leaf=30)
rfc.fit(AAPL_scaled_X,AAPL_y)
y_pred=rfc.predict(final_test)
y_pred
#accuracy_score(np.array(dji_y_test), y_pred)

array([0.])

# A function for easy prediction with any stocks

In [64]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

All variables can be obtained by the end of the trading day, which makes predicting the direction of the next trading day to be feasible. The approximate close price, SMA, etc can be obtained 10 mins before the market close. The data maybe a little inaccurate but most of the time the stock price won't have a volatile movement when it is close to end of trading day. This is not a good model for predicting the stock price after some big announcment since we would expect a very large movement in the post market.

In [178]:
class stock_direction_rf():
    def __init__(self, stock, start, end=date.today()
                 , test=None):
        self.stock = stock
        self.start = start
        self.end= end
        self.test=test
        
    def get_data(self):
        df= yf.download(self.stock, 
                start=self.start, 
                end=self.end, 
                progress=False)
        
        return df
        
    
    
    def clean_and_feature(self, df):
        
        
        #generate the percentage change of each day close price
        df['daily_close_change']=df['Close'].diff()/df['Close']
        #generate the direction of the stock move to each day
        momentum(df,self.stock,1)
        #shift the momentum in a negative direction for predicting the next day direction with today information
        df[str(self.stock)+'_momentum']=df[str(self.stock)+'_momentum'].shift(-1)
        #generate the SMA rate each day
        movingaverage(df, 10)
        movingaverage(df, 20)
        movingaverage(df, 50)
        #for the test day in the next day
        if self.test==None:
            next_day_test= df.iloc[-1].dropna().values.reshape(1,len(df.values[-1])-1)
        else:
            next_day_test=self.test
        df.dropna(inplace=True)
        return df, next_day_test
        
    def split_train_data(self, df):
        #Splitting the data into X and y, where df is the X data
        y=df[str(self.stock)+'_momentum']
        X=df.drop([str(self.stock)+'_momentum'], axis=1)
        #Scale the data with Min Max Scaler
        scaler=preprocessing.MinMaxScaler()
        scaler.fit(X)
        X=scaler.transform(X)
        
        return X, y
    
    def time_series_CV(self,X,y):
        tscv = TimeSeriesSplit(n_splits=5)
        i = 1
        score = []
        #cross validation for random forest to get the optimal hyperparameter
        for train_index, test_index in tscv.split(X):
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test =  X[train_index], X[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            criterion=['gini', 'entropy']
            for ne in np.linspace(50, 100, 6):
                for md in np.linspace(20, 40, 5):
                    for msl in np.linspace(30, 100, 8):
                        for c in criterion:
                            rfc = RandomForestClassifier(
                                criterion=c,
                                n_estimators=int(ne),
                                max_depth=int(md),
                                min_samples_leaf=int(msl))
                            rfc.fit(X_train, y_train)
                            score.append([i,
                                            c, 
                                            ne,
                                            md, 
                                            msl, 
                                            rfc.score(X_test, y_test)])
            i += 1
        
        #find the best parameter
        max_score=0
        parameter=[]
        for i in range(len(score)):
            if score[i][5]>=max_score:
                max_score=score[i][5]
                parameter=score[i]
        return score, parameter
        
    
    def predict(self, X, y, parameter, test):
        #Use the optimal hyperparameter obatained and train the all previous data except today data and use today data to
        #predict the stock direction in next trading date
        rfc = RandomForestClassifier(
                                criterion=parameter[1],
                                n_estimators=int(parameter[2]),
                                max_depth=int(parameter[3]),
                                min_samples_leaf=int(parameter[4]))
        
        rfc=rfc.fit(X,y)
        y_pred=rfc.predict(test)
        result=None
        if int(y_pred) ==1:
            result=print(str(self.stock)+ ' will close at a higher price in the next trading day')
        else:
            result=print(str(self.stock)+' will close at a lower price in the next trading dday')
            
        return y_pred, result
            
    def run_all(self, parameter=None):
        df=self.get_data()
        df, test_data=self.clean_and_feature(df)
        X,y=self.split_train_data(df)
        if parameter==None:
            score, parameter=self.time_series_CV(X,y)
        else:
            y_pred, result = self.predict(X,y, parameter, test_data)
        if parameter==None:
            return parameter, y_pred, result
        else:
            return y_pred, result
    
    


In [157]:

stock=stock_direction_rf('FUTU', '2016-01-01')
parameter, y_pred, result=stock.run_all()

FUTU will close at a higher price in the next trading day


In [162]:
parameter

[3, 'entropy', 100.0, 40.0, 100.0, 0.5970149253731343]

In [169]:
stock=stock_direction_rf('AAPL', '2016-01-01')
stock.run_all(parameter)

AAPL will close at a higher price in the next trading day


([5, 'gini', 80.0, 30.0, 50.0, 0.62], array([1.]), None)

In [183]:
parameter=[5, 'gini', 80.0, 30.0, 50.0, 0.62]
stock=stock_direction_rf('AAPL', '2016-01-01')
y_pred, result= stock.run_all(parameter)

AAPL will close at a higher price in the next trading day
